In [1]:
# Experiment Preparation

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

startdate_training = '2012-04-01'
enddate_training = '2020-05-01'

datasource = 'SPY'
data = yf.download(datasource,start = startdate_training,end = enddate_training)

[*********************100%***********************]  1 of 1 completed


In [2]:
def data_shift(data,target):
    data_shifted = data.drop([target],axis=1).shift()
    data_preprocessed = pd.concat([data_shifted,data[target]], axis=1)
    data_preprocessed.dropna(inplace=True)
    return data_preprocessed

In [3]:
data_preprocessed = data_shift(data,'Close')
data_preprocessed

,Open,High,Low,Adj Close,Volume,Close
Date,,,,,,
2012-04-03,140.639999,142.210007,140.360001,116.762001,151741100.0,141.259995
2012-04-04,141.639999,141.880005,140.429993,116.284531,155806700.0,139.860001
2012-04-05,140.220001,140.339996,139.339996,115.132057,146896000.0,139.789993
2012-04-09,139.380005,140.199997,139.259995,115.074432,137439400.0,138.220001
2012-04-10,138.029999,139.839996,137.839996,113.782028,127555900.0,135.899994
...,...,...,...,...,...,...
2020-04-24,280.489990,283.940002,278.750000,269.941437,104709700.0,282.970001
2020-04-27,280.730011,283.700012,278.500000,273.704071,85063200.0,287.049988
2020-04-28,285.119995,288.269989,284.619995,277.650482,77896600.0,285.730011


In [4]:
best_lag_list = [24, 20, 17, 11, 0]

In [5]:
def lag2shift(lag_list):
    shift_list = []
    max_lag = 30
    for i in lag_list:
        shift_list.append(max_lag - i)
    
    return shift_list

In [6]:
shift_list = lag2shift(best_lag_list)
shift_list

[6, 10, 13, 19, 30]

In [7]:
def rearrange(X,lags):
    Re = np.empty((X.shape[0],0))
    target = X.iloc[:,-1]
    for i in lags:
        Re = np.concatenate((Re, X.shift(i)), axis = 1)                 
    Re = np.concatenate((Re, target.to_frame(name = 'Target')), axis = 1)
    Re = pd.DataFrame(Re).dropna()
    return Re

In [8]:
data_var = rearrange(data_preprocessed,shift_list)
data_var

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
30,136.509995,137.559998,136.460007,112.860077,127765900.0,136.550003,139.789993,141.660004,139.630005,115.856499,...,114.103088,123884200.0,137.720001,140.639999,142.210007,140.360001,116.762001,151741100.0,141.259995,132.830002
31,136.279999,136.770004,134.919998,112.407295,213377700.0,135.740005,139.919998,140.460007,139.460007,115.510704,...,113.370422,198666700.0,137.949997,141.639999,141.880005,140.429993,116.284531,155806700.0,139.860001,130.860001
32,135.100006,136.610001,134.490005,111.740509,220752500.0,136.020004,140.339996,140.449997,138.990005,114.629951,...,113.559738,143199600.0,136.789993,140.220001,140.339996,139.339996,115.132057,146896000.0,139.789993,129.740005
33,136.679993,136.850006,135.710007,111.970978,150600000.0,135.610001,138.520004,139.300003,136.919998,112.777725,...,112.604820,171844900.0,137.309998,139.380005,140.199997,139.259995,115.074432,137439400.0,138.220001,131.970001
34,135.169998,136.869995,135.110001,111.633507,153032400.0,134.110001,136.509995,137.559998,136.460007,112.860077,...,113.032928,137484200.0,139.190002,138.029999,139.839996,137.839996,113.782028,127555900.0,135.899994,132.199997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027,277.570007,283.940002,275.459991,268.664642,121775000.0,279.100006,267.959991,276.000000,265.250000,265.056824,...,252.646927,257632800.0,253.419998,280.700012,281.940002,270.880005,263.824615,255316300.0,248.110001,282.970001
2028,279.149994,280.029999,275.760010,269.960815,131798300.0,286.640015,277.579987,281.200012,275.470001,269.090271,...,245.121674,224341200.0,261.649994,256.000000,266.660004,247.679993,238.582611,392220700.0,269.320007,287.049988
2029,285.380005,287.299988,282.399994,277.253876,146684800.0,281.589996,277.140015,277.510010,271.410004,266.633423,...,253.082169,171369500.0,257.750000,263.089996,271.480011,248.520004,258.978210,329566100.0,239.850006,285.730011
2030,282.609985,286.790009,281.350006,272.369232,100109300.0,273.040009,280.980011,284.899994,275.510010,274.497162,...,249.309891,194881100.0,246.149994,241.179993,256.899994,237.360001,230.639816,297240000.0,252.800003,293.209991


In [9]:
startdate_testing = '2020-05-01'
enddate_testing = '2022-06-01'
data_1 = yf.download(datasource,start = startdate_testing,end = enddate_testing)

[*********************100%***********************]  1 of 1 completed


In [10]:
data_preprocessed_1 = data_shift(data_1,'Close')

In [11]:
data_var_1 = rearrange(data_preprocessed_1,shift_list)
data_var_1

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
30,311.109985,313.000000,309.079987,301.164337,75794400.0,319.339996,302.459991,304.959991,299.470001,294.354889,...,276.905640,111146300.0,295.000000,291.709991,293.320007,288.589996,280.968140,122901700.0,282.790009,307.049988
31,317.230011,321.269989,317.160004,308.883118,150524700.0,323.200012,303.619995,306.209991,303.059998,295.544647,...,285.340179,119703800.0,291.970001,285.309998,290.660004,281.519989,273.529968,125180000.0,283.570007,312.959991
32,320.220001,323.410004,319.630005,312.616730,73641200.0,320.790009,306.549988,308.130005,305.100006,297.991760,...,282.409332,95189300.0,296.929993,280.739990,283.899994,279.130005,274.284424,80873200.0,286.190002,311.660004
33,320.299988,323.279999,319.359985,310.285645,77479200.0,319.000000,310.239990,313.220001,309.940002,301.957520,...,287.206909,85861700.0,294.880005,286.640015,289.250000,283.709991,276.818604,79569900.0,284.250000,311.779999
34,321.420013,322.390015,318.220001,308.554260,95000800.0,300.609985,311.109985,313.000000,309.079987,301.164337,...,285.224030,78293900.0,295.440002,288.040009,288.459991,283.779999,274.942108,73632600.0,287.679993,308.640015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,396.709991,403.179993,395.609985,399.992157,104174400.0,400.089996,405.100006,406.410004,396.500000,396.457428,...,414.310303,103996300.0,417.269989,447.970001,450.630005,445.940002,445.644958,79272700.0,439.920013,393.890015
521,399.980011,403.970001,397.600006,398.369171,78622400.0,408.320007,404.489990,406.079987,394.820007,397.373444,...,415.475250,122030000.0,427.809998,444.109985,445.000000,439.390015,438.027863,89770500.0,438.290009,397.369995
522,406.529999,408.570007,402.579987,406.563782,83029700.0,391.859985,398.070007,404.040009,391.959991,391.060730,...,425.969940,105449100.0,412.000000,443.079987,445.750000,436.649994,436.404877,84363600.0,443.309998,405.309998
523,403.500000,403.799988,390.549988,390.174530,117674500.0,389.459991,389.369995,395.799988,385.149994,390.652496,...,410.227936,145491100.0,414.480011,438.029999,444.109985,437.839996,441.403259,74070400.0,437.790009,415.260010


In [12]:
# 2002 + 495
# Covariance matrix

data_var_all = pd.concat([data_var,data_var_1], axis=0)

from pathlib import Path
filepath = Path('C:/Users/82422/OneDrive/internship/Optimal_Lag/Data/Data_VAR.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
data_var_all.to_csv(filepath)